#**Check for GPU Availability**

In [ ]:
import torch
print("Checking GPU availability...")
if torch.cuda.is_available():
    print(f"✅ GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("❌ No GPU detected! Please change runtime to GPU:")
    print("Runtime > Change runtime type > Hardware accelerator > GPU")
    raise RuntimeError("GPU not available")

# Install system dependencies
!apt -q install -y libgl1-mesa-glx

#**Clone ComfyUI & Install Requirements**

In [ ]:
import os, sys, subprocess

# ── 1) Reset to /workspace ──────────────────────────────────
try:
    os.chdir('/workspace')
except Exception:
    pass  # in case /workspace doesn't exist (it always does on Runpod)

# ── 2) Install cloudflared ─────────────────────────────────
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'cloudflared'], check=True)

# ── 3) Define paths ───────────────────────────────────────
COMFY = '/workspace/ComfyUI'

# ── 4) Check & abort if already cloned ───────────────────
if os.path.isdir(COMFY):
    print("❌ ERROR: '/workspace/ComfyUI' already exists.")
    print("👉 Remove it first with:\n   !rm -rf /workspace/ComfyUI")
    sys.exit(1)

# ── 5) Clone & install ────────────────────────────────────
subprocess.run(['git', 'clone', 'https://github.com/comfyanonymous/ComfyUI', COMFY], check=True)
os.chdir(COMFY)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)

print("✔ ComfyUI cloned and requirements installed.")


# Install ComfyUI Manager & Custom Nodes
Note: ComfyUI-Manager is essential for managing ComfyUI - Do not remove

In [ ]:
import os
import subprocess

# GitHub URLs - Add your custom nodes here
github_urls = [
    "https://github.com/ltdrdata/ComfyUI-Manager",  # Essential - DO NOT REMOVE
    # Add additional custom nodes below:

]

def clone_node(url):
    try:
        repo_name = url.split('/')[-1].replace('.git', '')
        dest = f"/workspace/ComfyUI/custom_nodes/{repo_name}"

        if os.path.exists(dest):
            print(f"⏭️  {repo_name} already exists")
            return

        subprocess.run(['git', 'clone', url, dest], check=True)
        print(f"✅ Installed {repo_name}")

        # Install requirements if present
        req_file = f"{dest}/requirements.txt"
        if os.path.exists(req_file):
            subprocess.run(['pip', 'install', '-r', req_file], check=True)

    except Exception as e:
        print(f"❌ Failed: {url.split('/')[-1]} - {e}")

# Create directory and install nodes
os.makedirs("/workspace/ComfyUI/custom_nodes", exist_ok=True)

print("📦 Installing custom nodes...")
for url in github_urls:
    if url.strip():
        clone_node(url.strip())

print("✨ Installation complete")

#**Download Models By ID (optional)**

In [ ]:
import os, re, sys, requests
from tqdm import tqdm

API_KEY = "API_KEY"  # ← set your CivitAI key
BASE     = "/workspace/ComfyUI/models"

# ─── Paste your URLs here ────────────────────────────────────────────────
ckpt_links = [
    "https://civitai.com/models/372465?modelVersionId=914390",
    "https://civitai.com/models/439889?modelVersionId=1199750",
    "https://civitai.com/models/795765/illustrious-xl"

]

lora_links = [
    "https://civitai.com/models/696418?modelVersionId=779327",
    "https://civitai.com/models/1327644?modelVersionId=1498957",
    "https://civitai.com/models/1333749/add-detail-slider",
    "https://civitai.com/models/669571/pony-add-more-details",
    "https://civitai.com/models/454172?modelVersionId=505635",
    "https://civitai.com/models/300005?modelVersionId=436219",
    "https://civitai.com/models/73756?modelVersionId=703107"

]

vae_links = [
    "https://civitai.com/models/199948?modelVersionId=224985"
]

embedding_links = [
    "https://civitai.com/models/118418?modelVersionId=134583",
    "https://civitai.com/models/583583?modelVersionId=651084"
]

controlnet_links = [
    # add ControlNet URLs here…
]
# ──────────────────────────────────────────────────────────────────────────

if not API_KEY:
    print("❌ Set your API_KEY"); sys.exit(1)

# prep folders
for folder in ("checkpoints","loras","vae","embeddings","controlnet"):
    os.makedirs(os.path.join(BASE, folder), exist_ok=True)

def parse_mid_and_override(url):
    m = re.search(r"models/(\d+)", url)
    if not m:
        raise ValueError(f"Bad URL: {url}")
    mid = m.group(1)
    over = re.search(r"modelVersionId=(\d+)", url)
    return mid, (over.group(1) if over else None)

def get_vid(mid):
    h = {"Authorization":f"Bearer {API_KEY}"}
    r = requests.get(f"https://civitai.com/api/v1/models/{mid}", headers=h, timeout=10)
    r.raise_for_status()
    return r.json()["modelVersions"][0]["id"]

def fetch_dl_info(vid):
    h = {"Authorization":f"Bearer {API_KEY}"}
    r = requests.get(f"https://civitai.com/api/v1/model-versions/{vid}", headers=h, timeout=10)
    r.raise_for_status()
    j = r.json()
    return j["downloadUrl"], j["files"][0]["name"]

def download(url, dest_folder):
    try:
        mid, override = parse_mid_and_override(url)
        vid = override or get_vid(mid)
        dl_url, fname = fetch_dl_info(vid)
    except Exception as e:
        print(f"✖ Error resolving {url}: {e}")
        return

    path = os.path.join(BASE, dest_folder, fname)
    r = requests.get(dl_url, headers={"Authorization":f"Bearer {API_KEY}"}, stream=True)
    if r.status_code != 200:
        print(f"✖ Download failed {fname}: HTTP {r.status_code}")
        return

    total = int(r.headers.get("content-length", 0))
    bar = tqdm(total=total, unit="iB", unit_scale=True, desc=fname)
    with open(path, "wb") as f:
        for chunk in r.iter_content(8192):
            f.write(chunk); bar.update(len(chunk))
    bar.close()
    print(f"✔ Saved → {path}")

# ─── Kick off downloads ───────────────────────────────────────────────────

print("Downloading checkpoints…")
for u in ckpt_links:      download(u, "checkpoints")

print("Downloading LoRAs…")
for u in lora_links:      download(u, "loras")

print("Downloading VAEs…")
for u in vae_links:       download(u, "vae")

print("Downloading embeddings…")
for u in embedding_links: download(u, "embeddings")

print("Downloading ControlNets…")
for u in controlnet_links: download(u, "controlnet")

print("\n✨ All done.")


# Download Model At Custom Location Via Civitai (Optional)

In [ ]:
import requests
import os

# Configuration
api_key = "API_KEY"  # ← set your CivitAI key
model_id = "116225"   # Replace with your Model ID
download_location = "/workspace/ComfyUI/models/upscale_models/"  # Replace it with your Location

# Create download directory if it doesn't exist
os.makedirs(download_location, exist_ok=True)

# Get model info
url = f"https://civitai.com/api/v1/models/{model_id}"
headers = {"Authorization": f"Bearer {api_key}"}

try:
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        model_data = response.json()
        download_url = model_data["modelVersions"][0]["files"][0]["downloadUrl"]
        filename = model_data["modelVersions"][0]["files"][0]["name"]
        
        print(f"Downloading: {filename}")
        print(f"From: {download_url}")
        
        # Download the file
        download_response = requests.get(download_url, headers=headers, stream=True)
        
        if download_response.status_code == 200:
            file_path = os.path.join(download_location, filename)
            
            with open(file_path, 'wb') as f:
                for chunk in download_response.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            print(f"Downloaded successfully to: {file_path}")
        else:
            print(f"Download failed: {download_response.status_code}")
            
    else:
        print(f"Error getting model info: {response.status_code} - {response.text}")
        
except Exception as e:
    print(f"Error: {e}")

# **Run ComfyUI and Access via Local Tunnel**

**Note:** ComfyUI lacks Gradio's `--share` option, so we use LocalTunnel to expose localhost:8188 to your browser.

## Why LocalTunnel via Node.js?

1. **Cloudflared:** Too slow and unreliable
2. **Ngrok & Pinggy:** Limited free usage, may require accounts  
3. **LocalTunnel:** Fast, reliable, no account needed, full control

**Benefits:** Zero configuration, custom subdomains, consistent performance.

In [ ]:
# Kill processes on port 8188 if it says port 8188 is already running or take too much time loading
# logs at /workspace/ComfyUI/comfyui.log
# logs at /workspace/ComfyUI/tunnel.log
!lsof -ti:8188 | xargs -r kill -9
!pkill -f "main.py"
print("✅ Port 8188 cleaned up")

In [ ]:
# Install Node.js silently in background
!curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash - > /dev/null 2>&1
!apt-get install -y nodejs > /dev/null 2>&1
!npm install -g localtunnel > /dev/null 2>&1

# Start ComfyUI
%cd /workspace/ComfyUI
!nohup python main.py --listen 0.0.0.0 --port 8188 > comfyui.log 2>&1 &
!sleep 10

# Start tunnel 
!nohup lt --port 8188 > tunnel.log 2>&1 &
!sleep 5

# Show URL and password on same line
!echo -n "Your URL is: "; cat tunnel.log | grep -o 'https://[^[:space:]]*'; echo -n "Tunnel Password: "; curl -s https://loca.lt/mytunnelpassword